# Imports

In [ ]:
from pylab import *
%matplotlib inline

caffe_library = '../caffelibrary/'

import sys
sys.path.insert(0, caffe_library)
import caffe
import os

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn import preprocessing
import scipy.misc

# Methods

In [ ]:
scaler = preprocessing.MinMaxScaler()

def scaleimage(image):    
    ascolumns = image.reshape(-1, 3)
    t = scaler.fit_transform(ascolumns)
    transformed = t.reshape(image.shape)     
        
    return transformed


def createcaffeinput(imagepath):
    # Open the image
    image = np.array(Image.open(imagepath))
    # Scale (0 - 1)
    image = scaleimage(image)
    '''
    Needed for caffe. For example, if the image has shape [720, 480, 3], this
    will transform it in to the shape [3, 720, 480]
    '''
    image = image.transpose(2, 0, 1)
    # Finally, we need to add a new axis, this will make the shape of the object: [1, 3, 720, 480]
    image = image[np.newaxis, :, :]
    
    return image


def inputimage(net, layername, imagepath):
    caffeinput = createcaffeinput(imagepath)
    net.blobs[layername].reshape(*caffeinput.shape)
    net.blobs[layername].data[...] = caffeinput
    
    
def saveresults(net, layers):
    for name in layers:
        image = net.blobs[name].data[0]
        scipy.misc.imsave(name + '.jpg', image.transpose(1, 2, 0))
        

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

def getdistance(x, y):
    return np.linalg.norm(x-y)

# Input Images

In [ ]:
IMAGE_FILE = 'Some path' # label
IMAGE_FILE_DIRTY = 'Some path' #processed

# Loading the caffe model

In [ ]:
MODEL_ARQUITECTURE = 'prototxt file path'
MODEL = 'Path to a caffe model'
net = caffe.Net(MODEL_ARQUITECTURE, MODEL, caffe.TEST)

# Test

In [ ]:
inputimage(net, 'clean', IMAGE_FILE)
inputimage(net, 'dirty', IMAGE_FILE_DIRTY)

out = net.forward()
saveresults(net, ['clean', 'dirty', 'result'])

print out